In [1]:
import os
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.figure_factory as ff

## Define Term Dates

In [2]:
# Define the dates
number_of_terms=3
term_start_dates=['28/09/2020','11/01/2021','26/04/2021']
term_end_dates=['18/12/2020','26/03/2021','11/06/2021']
reading_week_dates=['09/11/2020','15/02/2021']
# Convert the string to dates
term_start_dates=[pd.to_datetime(date, format='%d/%m/%Y') for date in term_start_dates]
term_end_dates=[pd.to_datetime(date, format='%d/%m/%Y') for date in term_end_dates]
reading_week_dates=[pd.to_datetime(date, format='%d/%m/%Y') for date in reading_week_dates]

## Create the Calendar DataFrame

In [3]:
def get_term_number(df):
    if (df['Date']>=reading_week_dates[0]) & (df['Date']<=(reading_week_dates[0] + pd.Timedelta(6,unit='days'))):
        return 'Reading Week 1'
    elif (df['Date']>=reading_week_dates[1]) & (df['Date']<=(reading_week_dates[1] + pd.Timedelta(6,unit='days'))):
        return 'Reading Week 2'
    if (df['Date']>=term_start_dates[0]) & (df['Date']<=term_end_dates[0]):
        return 'Term 1'
    elif (df['Date']>term_end_dates[0]) & (df['Date']<term_start_dates[1]):
        return 'Christmas Holidays'
    elif (df['Date']>=term_start_dates[1]) & (df['Date']<=term_end_dates[1]):
        return 'Term 2'
    elif( df['Date']>term_end_dates[1]) & (df['Date']<term_start_dates[2]):
        return 'Easter Holidays'
    elif (df['Date']>=term_start_dates[2]) & (df['Date']<=term_end_dates[2]):
        return 'Term 3'


In [4]:
calendar=pd.DataFrame(columns=['Date'])
calendar['Date']=pd.date_range(start=term_start_dates[0], end=term_end_dates[number_of_terms-1])
if os.name=='nt':
    calendar['Date_F']=calendar['Date'].dt.strftime('%#d %B %Y')
else:
    calendar['Date_F']=calendar['Date'].dt.strftime('%-d %B %Y')
calendar['Day'] =calendar['Date'].dt.day
calendar['Week Day']=calendar['Date'].dt.day_name()
calendar['Week Day'] = pd.Categorical(calendar['Week Day'], categories=
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],
    ordered=True)
calendar['Academic_Week_Number'] = calendar['Date'].apply(lambda x: (x.isocalendar()[1]-35) if (x.isocalendar()[1]-35)>0 else (x.isocalendar()[1]+18))

In [5]:
calendar['Term Class']=calendar.apply(get_term_number,axis=1)

## Add Planned Hours

In [6]:
calendar['Planned_work_hours']=5

## Add number of Hours required for Coursework

In [7]:
calendar['Hours_required_for_Coursework']=1

In [8]:
# Adding contingency of 1.2

## Calculate the Workload

In [9]:
calendar['Workload']=(calendar['Hours_required_for_Coursework']/calendar['Planned_work_hours'])

In [10]:
calendar.to_csv('calendar.csv')

## Get the HeatMap

In [11]:
calendar[calendar['Date'].dt.month==9].head()

,Date,Date_F,Day,Week Day,Academic_Week_Number,Term Class,Planned_work_hours,Hours_required_for_Coursework,Workload
0,2020-09-28,28 September 2020,28,Monday,5,Term 1,5,1,0.2
1,2020-09-29,29 September 2020,29,Tuesday,5,Term 1,5,1,0.2
2,2020-09-30,30 September 2020,30,Wednesday,5,Term 1,5,1,0.2


In [12]:
academic_year = list(range(9,13)) + list(range(1,7))

In [13]:
monthly=calendar[calendar['Date'].dt.month==10].pivot(index='Academic_Week_Number', columns='Week Day', values='Workload')

In [14]:
yearly=calendar.pivot(index='Week Day', columns='Academic_Week_Number', values='Workload')

In [15]:
monthly

Week Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Academic_Week_Number,,,,,,,
5,NaN,NaN,NaN,0.2,0.2,0.2,0.2
6,0.2,0.2,0.2,0.2,0.2,0.2,0.2
7,0.2,0.2,0.2,0.2,0.2,0.2,0.2
8,0.2,0.2,0.2,0.2,0.2,0.2,0.2
9,0.2,0.2,0.2,0.2,0.2,0.2,NaN


In [16]:
yearly

Academic_Week_Number,5,6,7,8,9,10,11,12,13,14,...,32,33,34,35,36,37,38,39,40,41
Week Day,,,,,,,,,,,,,,,,,,,,,
Monday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
Tuesday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
Wednesday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
Thursday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
Friday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
Saturday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,NaN
Sunday,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,NaN


In [17]:
annotations_monthly=calendar[calendar['Date'].dt.month==10].pivot(index='Academic_Week_Number', columns='Week Day', values='Day')

In [18]:
annotations_yearly=calendar.pivot(index='Week Day', columns='Academic_Week_Number', values='Day')

In [19]:
dates_monthly=calendar[calendar['Date'].dt.month==10].pivot(index='Academic_Week_Number', columns='Week Day', values='Date_F')

In [20]:
dates_yearly = calendar.pivot(index='Week Day', columns='Academic_Week_Number', values='Date_F')

## Month wise

In [21]:
def generate_heatmap(df, dates, title='', autosize=True, width=None, height=None, x_label='', y_label='', side="bottom"):

    data = go.Heatmap(z=df,
                      x=df.columns,
                      y=df.index,
                      hovertext=dates,
                      ygap=3,
                      xgap=3,
                      hoverongaps = False,
                      showscale = False,
                      colorscale=['rgb(0,100,0)', 'rgb(100,0,0)'], zmax=1.5, zmid=0.5, zmin=0,
                      hovertemplate = '%{hovertext}<br>Workload: <b>%{z:%}</b><extra></extra>')
                      
    layout = go.Layout(autosize=autosize,
                       width=width,
                       height=height,
                       title="<b>{}</b>".format(title),
                       title_x=0.5,
                       xaxis_title='<b>{}</b>'.format(x_label),
                       yaxis_title='<b>{}</b>'.format(y_label),
                       template="simple_white",
                       xaxis=dict(side=side, showline=False,ticks=''),
                       yaxis=dict(autorange="reversed", showline=False,ticks=''),
                       margin=dict(pad=5),
                       showlegend=False, 
                       hoverlabel=dict(align = 'auto',bordercolor='rgb(48,48,48)',bgcolor='rgb(48,48,48)',font=dict(color='white')))

    fig = go.Figure(data=data, layout=layout)
    return fig
                    

In [22]:
def generate_month_wise(monthly, annotations, dates_monthly): 
    fig=generate_heatmap(monthly,dates_monthly,title='October', y_label='Academic Week Number', side="top")
    for i in np.arange(len(monthly.columns)):
        for j in np.arange(len(monthly.index)):
            if not np.isnan(annotations.iloc[j,i]):
                fig.add_annotation(x=monthly.columns[i],y=monthly.index[j],text=int((annotations.iloc[j,i])),
                                   font_color='white',showarrow=False)
    fig.show()


In [23]:
generate_month_wise(monthly,annotations_monthly,dates_monthly)

## Year wise

In [24]:
def generate_yearwise(yearly,annotations,dates_yearly):
    fig=generate_heatmap(yearly,dates_yearly, autosize=False, width=1200, height=400, x_label='Academic Week Number')
    fig.show()

In [25]:
generate_yearwise(yearly, annotations_yearly,dates_yearly)

## Term wise

### matplotlib version
```python3
import matplotlib.pyplot as plt

def heatmap(data, ax=None):

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data)

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    
    # Label them with the respective list entries.
    ax.set_xticklabels(data.columns)
    ax.set_yticklabels(data.index)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(bottom=False, top=False, left=False,right=False,
                   labeltop=True, labelbottom=False)


    # Make spine white and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_color('w')
        spine.set_linewidth(4)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im

# Show colour bar at the Bottom Separately
# cbar = ax.figure.colorbar(im, ax=ax)
# cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

fig, ax = plt.subplots(figsize=(10,6))
heatmap(monthly,ax)
```